In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import librosa
from tqdm import tqdm_notebook as tqdm
import json
import pandas as pd
import os
import h5py
import pickle
import numpy.linalg as la
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler

In [2]:
param_grid = {'C': [10**k for k in range(-10, 4, 1)]}  
scoring = 'roc_auc'; cv = 3

# Load embeddings and align classes

In [3]:
embedding = 'openl3'
debias_method = '-lda-genre'

In [4]:
# load embeddings
embeddings = h5py.File('embeddings.h5', 'r')

def printname(name):
    print(name)
embeddings.visit(printname)

irmas
irmas/openl3
irmas/openl3/features
irmas/openl3/keys
irmas/vggish
irmas/vggish/features
irmas/vggish/keys
irmas/yamnet
irmas/yamnet/features
irmas/yamnet/keys
openmic
openmic/openl3
openmic/openl3/features
openmic/openl3/keys
openmic/vggish
openmic/vggish/features
openmic/vggish/keys
openmic/yamnet
openmic/yamnet/features
openmic/yamnet/keys


In [5]:
with open('class-map-10.json', 'r') as f: # only consider 10 classes of Openmic dataset
    class_map = json.load(f)
    
class_map

{'cello': 3,
 'clarinet': 4,
 'flute': 7,
 'guitar': 8,
 'organ': 11,
 'piano': 12,
 'saxophone': 13,
 'trumpet': 16,
 'violin': 18,
 'voice': 19}

In [6]:
# use a dict to align the classes between Openmic dataset (key) and Irmas dataset (val)
class_align = {'cello': 'cel',
               'clarinet': 'cla',
               'flute': 'flu',
               'guitar': ['gac', 'gel'],
               'organ': 'org',
               'piano': 'pia',
               'saxophone': 'sax',
               'trumpet': 'tru',
               'violin': 'vio',
               'voice': 'voi'}

In [7]:
# use a Pandas DataFrame to record all results and save into a csv file later
result_all = pd.DataFrame({'instrument': [],
                          'train_set': [],
                          'test_set': [],
                          'precision': [],
                          'recall': [],
                          'f1-score': [],
                          'support': [],
                          'accuracy': [],
                          'roc_auc': [],
                          'ap': [],
                          'embedding': [],
                         })

# irmas->irmas

In [8]:
# irmas: Vggish embedding
feature = np.array(embeddings['irmas'][embedding]['features'])
keys_ori = np.array(embeddings['irmas'][embedding]['keys'])
print(feature.shape, keys_ori.shape)

key_clip = np.unique(keys_ori)
print(key_clip.shape)

feature_clip = []

for key in tqdm(key_clip):
    feature_clip.append(np.mean(feature[keys_ori[:]==key,:],axis=0))
    
feature_clip = np.array(feature_clip)
key_clip = np.array([str(k, 'utf-8') for k in key_clip])
print(feature_clip.shape, key_clip.shape)

(167625, 512) (167625,)
(6705,)


  0%|          | 0/6705 [00:00<?, ?it/s]

(6705, 512) (6705,)


In [9]:
key_train = set(pd.read_csv('irmas_train.csv', header=None, squeeze=True))
key_test = set(pd.read_csv('irmas_test.csv', header=None, squeeze=True))

In [10]:
# these loops go through all sample keys, and save their row numbers to either idx_train or idx_test
idx_train, idx_test = [], []

for k in range(len(key_clip)):
    if str(key_clip[k]) in key_train:
        idx_train.append(k)
    elif str(key_clip[k]) in key_test:
        idx_test.append(k)
    else:
        # This should never happen, but better safe than sorry.
        raise RuntimeError('Unknown sample key={}! Abort!'.format(str(key_clip[k])))
        
# cast the idx_* arrays to numpy structures
idx_train = np.asarray(idx_train)
idx_test = np.asarray(idx_test)

In [11]:
#keys = np.array([str(k, encoding='utf-8') for k in key_clip])
keys = np.array(key_clip)
keys = [key[key.index('[')+1:key.index(']')] for key in keys]

for key in class_align:
    keys = [key if x in class_align[key] else x for x in keys]
    
keys = np.array(keys)
np.unique(keys)

array(['cello', 'clarinet', 'flute', 'guitar', 'organ', 'piano',
       'saxophone', 'trumpet', 'violin', 'voice'], dtype='<U9')

In [12]:
# use the split indices to partition the features, labels, and masks
X_train = feature_clip[idx_train,:]
X_test = feature_clip[idx_test]

Y_true_train = keys[idx_train]
Y_true_test = keys[idx_test]

# print out the sliced shapes as a sanity check
print(X_train.shape)
print(X_test.shape)

(5039, 512)
(1666, 512)


In [13]:
# standardize embedding
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

############################# project the LDA direction #############################
file = open('models/datasep_' + embedding + debias_method + '.pickle', 'rb')
globals()['LDAcoef_' + embedding] = pickle.load(file)
file.close()

W = globals()['LDAcoef_' + embedding].copy()
U, s, V = la.svd(W, full_matrices=False)
A = np.dot(V.T, V)
print(W.shape, U.shape, s.shape, V.shape, A.shape)

Xdeb_train = X_train.dot(np.eye(len(A)) - A)
Xdeb_test = X_test.dot(np.eye(len(A)) - A)
print(Xdeb_train.shape, Xdeb_test.shape)

X_train = Xdeb_train; X_test = Xdeb_test; del(Xdeb_train, Xdeb_test)
print(X_train.shape, X_test.shape)

(5, 512) (5, 5) (5,) (5, 512) (512, 512)
(5039, 512) (1666, 512)
(5039, 512) (1666, 512)


In [14]:
train_set_name, test_set_name = 'irmas', 'irmas'

# use a dictionary to include the classifier for each instrument trained on the dataset based on the embedding
globals()['models_'+train_set_name] = dict()  

# iterate over all istrument classes, and fit a model for each one
for instrument in class_align:
    
    # get the training and testing labels for each instrument
    Y_true_train_inst = Y_true_train==instrument
    Y_true_test_inst = Y_true_test==instrument
    
    # initialize and a logistic regression model
    LRmodel = LogisticRegression(random_state=0, penalty='l2', solver='liblinear', class_weight='balanced')
    
    # hyperparameter tunning for logistic regression model
    clf =  GridSearchCV(LRmodel, param_grid=param_grid, cv=cv, scoring=scoring)  
    
    # fit the model
    clf.fit(X_train, Y_true_train_inst)
    
    # predict
    Y_pred_test_inst = clf.predict(X_test)
    
    # Get prediction scores for the positive class
    Y_pred_test_scores = clf.predict_proba(X_test)[:, 1]
    
    # print result for each instrument
    print('-' * 52); print(instrument); print('\tTEST')
    print(classification_report(Y_true_test_inst, Y_pred_test_inst))
    
    model_auc = roc_auc_score(Y_true_test_inst, Y_pred_test_scores)
    model_ap = average_precision_score(Y_true_test_inst, Y_pred_test_scores)
    print(f'ROC-AUC = {model_auc:.3f}\t\tAP = {model_ap:.3f}')
    
    # store the classifier in the model dictionary
    globals()['models_'+train_set_name][instrument] = clf
    
    # record the result for each instrument
    report = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['True']
    report['roc_auc'] = model_auc
    report['ap'] = model_ap
    
    report_accuracy = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['accuracy'][-2]
    result_inst = [instrument, train_set_name, test_set_name, report['precision'], report['recall'],
                   report['f1-score'], report['support'], report_accuracy, model_auc, model_ap, embedding + debias_method]
    result_all = result_all.append(pd.DataFrame(np.expand_dims(np.array(result_inst), axis=0), 
                                                columns=result_all.columns), ignore_index=True)
    
with open('models/models_' + train_set_name + '_' + embedding + debias_method + '.pickle', 'wb') as fdesc:
    pickle.dump(globals()['models_'+train_set_name], fdesc)

----------------------------------------------------
cello
	TEST
              precision    recall  f1-score   support

       False       0.99      0.95      0.97      1572
        True       0.50      0.81      0.62        94

    accuracy                           0.94      1666
   macro avg       0.74      0.88      0.79      1666
weighted avg       0.96      0.94      0.95      1666

ROC-AUC = 0.955		AP = 0.748
----------------------------------------------------
clarinet
	TEST
              precision    recall  f1-score   support

       False       0.98      0.92      0.95      1539
        True       0.44      0.74      0.55       127

    accuracy                           0.91      1666
   macro avg       0.71      0.83      0.75      1666
weighted avg       0.94      0.91      0.92      1666

ROC-AUC = 0.910		AP = 0.686
----------------------------------------------------
flute
	TEST
              precision    recall  f1-score   support

       False       0.97      0.92    

# openmic->openmic

In [15]:
feature = np.array(embeddings['openmic'][embedding]['features'])
keys = np.array(embeddings['openmic'][embedding]['keys'])
print(feature.shape, keys.shape)

key_clip = np.unique(keys)

feature_clip = []

for key in tqdm(key_clip):
    feature_clip.append(np.mean(feature[keys[:]==key,:],axis=0))
    
feature_clip = np.array(feature_clip)
print(feature_clip.shape, key_clip.shape)

key_clip = np.array([str(k, 'utf-8') for k in key_clip])

(1900000, 512) (1900000,)


  0%|          | 0/20000 [00:00<?, ?it/s]

(20000, 512) (20000,)


In [16]:
# key-label map using the information from the dataset source
data_root = 'openmic-2018/'

np_load_old = np.load   # save np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True,**k)   # modify the default parameters of np.load

Ytrue = np.load(os.path.join(data_root, 'openmic-2018.npz'))['Y_true']
Ymask = np.load(os.path.join(data_root, 'openmic-2018.npz'))['Y_mask']
sample_key = np.load(os.path.join(data_root, 'openmic-2018.npz'))['sample_key']

np.load = np_load_old   # restore np.load for future normal usage
del(np_load_old)

print(Ytrue.shape, Ymask.shape, sample_key.shape)

(20000, 20) (20000, 20) (20000,)


In [17]:
Y_true = []
Y_mask = []

for key in tqdm(key_clip):
    Y_true.append(Ytrue[sample_key==key])
    Y_mask.append(Ymask[sample_key==key])
    
Y_true = np.squeeze(np.array(Y_true))
Y_mask = np.squeeze(np.array(Y_mask))

print(feature_clip.shape, Y_true.shape, Y_mask.shape)

  0%|          | 0/20000 [00:00<?, ?it/s]

(20000, 512) (20000, 20) (20000, 20)


In [18]:
# train-test split
train_set = set(pd.read_csv(data_root + 'openmic2018_train.csv', header=None, squeeze=True))
test_set = set(pd.read_csv(data_root + 'openmic2018_test.csv', header=None, squeeze=True))
print('# Train: {},  # Test: {}'.format(len(train_set), len(test_set)))

# Train: 14915,  # Test: 5085


In [19]:
idx_train, idx_test = [], []

for idx, n in enumerate(key_clip):
    if n in train_set:
        idx_train.append(idx)
    elif n in test_set:
        idx_test.append(idx)
    else:
        raise RuntimeError('Unknown sample key={}! Abort!'.format(key_clip[n]))
        
idx_train = np.asarray(idx_train)
idx_test = np.asarray(idx_test)

In [20]:
X_train = feature_clip[idx_train]
X_test = feature_clip[idx_test]

Y_true_train = Y_true[idx_train]
Y_true_test = Y_true[idx_test]

Y_mask_train = Y_mask[idx_train]
Y_mask_test = Y_mask[idx_test]

print(X_train.shape); print(X_test.shape)

(14915, 512)
(5085, 512)


In [21]:
# standardize embedding
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Xdeb_train = X_train.dot(np.eye(len(A)) - A)
Xdeb_test = X_test.dot(np.eye(len(A)) - A)
print(Xdeb_train.shape, Xdeb_test.shape)

X_train = Xdeb_train; X_test = Xdeb_test; del(Xdeb_train, Xdeb_test)
print(X_train.shape, X_test.shape)

(14915, 512) (5085, 512)
(14915, 512) (5085, 512)


In [22]:
# This part of the code follows the baseline model for instrument recognition on the openmic dataset:
# https://github.com/cosmir/openmic-2018/blob/master/examples/modeling-baseline.ipynb
train_set_name, test_set_name = 'openmic', 'openmic'

# use a dictionary to include the classifier for each instrument trained on the dataset based on the embedding
globals()['models_'+train_set_name] = dict()  

# We'll iterate over all istrument classes, and fit a model for each one
# After training, we'll print a classification report for each instrument
for instrument in class_align:
    
    # Map the instrument name to its column number
    inst_num = class_map[instrument]
    
    # First, sub-sample the data: we need to select down to the data for which we have annotations
    # This is what the mask arrays are for
    train_inst = Y_mask_train[:, inst_num]
    test_inst = Y_mask_test[:, inst_num]
    
    # Here, we're using the Y_mask_train array to slice out only the training examples
    # for which we have annotations for the given class
    X_train_inst = X_train[train_inst]
    
    # Again, we slice the labels to the annotated examples
    # We thresold the label likelihoods at 0.5 to get binary labels
    Y_true_train_inst = Y_true_train[train_inst, inst_num] >= 0.5
    
    # Repeat the above slicing and dicing but for the test set
    X_test_inst = X_test[test_inst]
    Y_true_test_inst = Y_true_test[test_inst, inst_num] >= 0.5

    # initialize and a logistic regression model
    LRmodel = LogisticRegression(random_state=0, penalty='l2', solver='liblinear', class_weight='balanced')
    
    # hyperparameter tunning for logistic regression model
    clf =  GridSearchCV(LRmodel, param_grid=param_grid, cv=cv, scoring=scoring)    
    
    # fit the model
    clf.fit(X_train_inst, Y_true_train_inst)

    # predict
    Y_pred_test_inst = clf.predict(X_test_inst)
    # Get prediction scores for the positive class
    Y_pred_test_scores = clf.predict_proba(X_test_inst)[:, 1]
    
    # print result for each instrument
    print('-' * 52); print(instrument); print('\tTEST')
    print(classification_report(Y_true_test_inst, Y_pred_test_inst))
    
    model_auc = roc_auc_score(Y_true_test_inst, Y_pred_test_scores)
    model_ap = average_precision_score(Y_true_test_inst, Y_pred_test_scores)
    print(f'ROC-AUC = {model_auc:.3f}\t\tAP = {model_ap:.3f}')
    
    # store the classifier in the model dictionary
    globals()['models_'+train_set_name][instrument] = clf
    
    # record the result for each instrument
    report = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['True']
    report['roc_auc'] = model_auc
    report['ap'] = model_ap
    
    report_accuracy = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['accuracy'][-2]
    result_inst = [instrument, train_set_name, test_set_name, report['precision'], report['recall'],
                   report['f1-score'], report['support'], report_accuracy, model_auc, model_ap, embedding + debias_method]
    result_all = result_all.append(pd.DataFrame(np.expand_dims(np.array(result_inst), axis=0), 
                                                columns=result_all.columns), ignore_index=True)
    
with open('models/models_' + train_set_name + '_' + embedding + debias_method + '.pickle', 'wb') as fdesc:
    pickle.dump(globals()['models_'+train_set_name], fdesc)

----------------------------------------------------
cello
	TEST
              precision    recall  f1-score   support

       False       0.85      0.75      0.80       259
        True       0.75      0.85      0.79       226

    accuracy                           0.80       485
   macro avg       0.80      0.80      0.80       485
weighted avg       0.80      0.80      0.80       485

ROC-AUC = 0.886		AP = 0.854
----------------------------------------------------
clarinet
	TEST
              precision    recall  f1-score   support

       False       0.91      0.58      0.71       503
        True       0.34      0.79      0.48       137

    accuracy                           0.63       640
   macro avg       0.63      0.69      0.59       640
weighted avg       0.79      0.63      0.66       640

ROC-AUC = 0.755		AP = 0.428
----------------------------------------------------
flute
	TEST
              precision    recall  f1-score   support

       False       0.86      0.58    

# irmas->openmic

In [23]:
train_set_name, test_set_name = 'irmas', 'openmic' 

# iterate over all istrument classes, and fit a model for each one
for instrument in class_align:
    
    # Map the instrument name to its column number
    inst_num = class_map[instrument]
    
    # First, sub-sample the data: we need to select down to the data for which we have annotations 
    # This is what the mask arrays are for
    test_inst = Y_mask_test[:, inst_num]

    # Repeat the above slicing and dicing but for the test set
    X_test_inst = X_test[test_inst]
    Y_true_test_inst = Y_true_test[test_inst, inst_num] >= 0.5

    # evaluate the classifier 
    Y_pred_test_inst =  globals()['models_'+train_set_name][instrument].predict(X_test_inst)
    Y_pred_test_scores =  globals()['models_'+train_set_name][instrument].predict_proba(X_test_inst)[:, 1]

    # print result for each instrument
    print('-' * 52); print(instrument); print('\tTEST')
    print(classification_report(Y_true_test_inst, Y_pred_test_inst))
    
    model_auc = roc_auc_score(Y_true_test_inst, Y_pred_test_scores)
    model_ap = average_precision_score(Y_true_test_inst, Y_pred_test_scores)
    print(f'ROC-AUC = {model_auc:.3f}\t\tAP = {model_ap:.3f}')
    
    # record the result for each instrument
    report = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['True']
    report['roc_auc'] = model_auc
    report['ap'] = model_ap
    
    report_accuracy = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['accuracy'][-2]
    result_inst = [instrument, train_set_name, test_set_name, report['precision'], report['recall'],
                   report['f1-score'], report['support'], report_accuracy, model_auc, model_ap, embedding + debias_method]
    result_all = result_all.append(pd.DataFrame(np.expand_dims(np.array(result_inst), axis=0), \
                                                columns=result_all.columns), ignore_index=True)

----------------------------------------------------
cello
	TEST
              precision    recall  f1-score   support

       False       0.59      0.82      0.69       259
        True       0.63      0.34      0.44       226

    accuracy                           0.60       485
   macro avg       0.61      0.58      0.56       485
weighted avg       0.61      0.60      0.57       485

ROC-AUC = 0.699		AP = 0.619
----------------------------------------------------
clarinet
	TEST
              precision    recall  f1-score   support

       False       0.81      0.78      0.79       503
        True       0.28      0.31      0.30       137

    accuracy                           0.68       640
   macro avg       0.54      0.55      0.55       640
weighted avg       0.69      0.68      0.69       640

ROC-AUC = 0.656		AP = 0.289
----------------------------------------------------
flute
	TEST
              precision    recall  f1-score   support

       False       0.76      0.86    

# openmic->irmas

In [24]:
# irmas: vggish embedding
feature = np.array(embeddings['irmas'][embedding]['features'])
keys_ori = np.array(embeddings['irmas'][embedding]['keys'])
print(feature.shape, keys_ori.shape)

key_clip = np.unique(keys_ori)
print(key_clip.shape)

feature_clip = []

for key in tqdm(key_clip):
    feature_clip.append(np.mean(feature[keys_ori[:]==key,:],axis=0))
    
feature_clip = np.array(feature_clip)
key_clip = np.array([str(k, encoding='utf-8') for k in key_clip])
print(feature_clip.shape, key_clip.shape)

(167625, 512) (167625,)
(6705,)


  0%|          | 0/6705 [00:00<?, ?it/s]

(6705, 512) (6705,)


In [25]:
key_train = list(pd.read_csv('irmas_train.csv', header=None, squeeze=True))
key_test = list(pd.read_csv('irmas_test.csv', header=None, squeeze=True))

idx_train, idx_test = [], []

for k in range(len(key_clip)):
    if str(key_clip[k]) in key_train:
        idx_train.append(k)
    elif str(key_clip[k]) in key_test:
        idx_test.append(k)
    else:
        raise RuntimeError('Unknown sample key={}! Abort!'.format(key_clip[k]))
        
idx_train = np.asarray(idx_train)
idx_test = np.asarray(idx_test)

In [26]:
#keys = np.array([str(k, 'utf-8') for k in key_clip])
keys = np.array(key_clip)
keys = [key[key.index('[')+1:key.index(']')] for key in keys]

for key in class_align:
    keys = [key if x in class_align[key] else x for x in keys]
    
keys = np.array(keys)
np.unique(keys)

array(['cello', 'clarinet', 'flute', 'guitar', 'organ', 'piano',
       'saxophone', 'trumpet', 'violin', 'voice'], dtype='<U9')

In [27]:
X_train = feature_clip[idx_train,:]
X_test = feature_clip[idx_test]

Y_true_train = keys[idx_train]
Y_true_test = keys[idx_test]

print(X_train.shape)
print(X_test.shape)

(5039, 512)
(1666, 512)


In [28]:
# standardize embedding
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Xdeb_train = X_train.dot(np.eye(len(A)) - A)
Xdeb_test = X_test.dot(np.eye(len(A)) - A)
print(Xdeb_train.shape, Xdeb_test.shape)

X_train = Xdeb_train; X_test = Xdeb_test; del(Xdeb_train, Xdeb_test)
print(X_train.shape, X_test.shape)

(5039, 512) (1666, 512)
(5039, 512) (1666, 512)


In [29]:
train_set_name, test_set_name = 'openmic', 'irmas' 

# iterate over all istrument classes, and fit a model for each one
for instrument in class_align:
    
    # get the training and testing labels for each instrument
    Y_true_test_inst = Y_true_test==instrument

    # evaluate the classifier
    Y_pred_test_inst =  globals()['models_'+train_set_name][instrument].predict(X_test)
    Y_pred_test_scores =  globals()['models_'+train_set_name][instrument].predict_proba(X_test)[:, 1]
    
    # print result for each instrument
    print('-' * 52); print(instrument); print('\tTEST')
    print(classification_report(Y_true_test_inst, Y_pred_test_inst))
    
    model_auc = roc_auc_score(Y_true_test_inst, Y_pred_test_scores)
    model_ap = average_precision_score(Y_true_test_inst, Y_pred_test_scores)
    print(f'ROC-AUC = {model_auc:.3f}\t\tAP = {model_ap:.3f}')
    
    # record the result for each instrument
    report = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['True']
    report['roc_auc'] = model_auc
    report['ap'] = model_ap
    
    report_accuracy = pd.DataFrame(classification_report(Y_true_test_inst, Y_pred_test_inst, output_dict=True))['accuracy'][-2]
    result_inst = [instrument, train_set_name, test_set_name, report['precision'], report['recall'],
                   report['f1-score'], report['support'], report_accuracy, model_auc, model_ap, embedding + debias_method]
    result_all = result_all.append(pd.DataFrame(np.expand_dims(np.array(result_inst), axis=0), \
                                                columns=result_all.columns), ignore_index=True)

----------------------------------------------------
cello
	TEST
              precision    recall  f1-score   support

       False       0.98      0.63      0.77      1572
        True       0.12      0.82      0.20        94

    accuracy                           0.64      1666
   macro avg       0.55      0.72      0.49      1666
weighted avg       0.93      0.64      0.74      1666

ROC-AUC = 0.799		AP = 0.239
----------------------------------------------------
clarinet
	TEST
              precision    recall  f1-score   support

       False       0.98      0.69      0.81      1539
        True       0.18      0.81      0.29       127

    accuracy                           0.70      1666
   macro avg       0.58      0.75      0.55      1666
weighted avg       0.92      0.70      0.77      1666

ROC-AUC = 0.813		AP = 0.244
----------------------------------------------------
flute
	TEST
              precision    recall  f1-score   support

       False       0.97      0.61    

# Plot result on each instrument

In [30]:
# save all result into a csv file
result_all.to_csv('results/result_' + embedding + debias_method + '.csv', index=False)

C_vggish = []

for inst in class_align:
    C_vggish.append(models_irmas[inst].best_estimator_.C)
    C_vggish.append(models_openmic[inst].best_estimator_.C)
    
print(max(C_vggish), min(C_vggish))

0.1 1e-05
